#### Convolutional Neural Network (CNN) for classifying the CIFAR-10 Dataset

In [8]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [92]:
num_epochs = 1
batch_size = 10
learning_rate = 0.0001

In [11]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [1.0, 1.0, 1.0])]
)

In [12]:
train_dataset = torchvision.datasets.CIFAR10(root='./datasets', download=True, transform=transform, train=True)
test_dataset = torchvision.datasets.CIFAR10(root='./datasets', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, shuffle=False, batch_size=batch_size)

100%|██████████| 170M/170M [01:56<00:00, 1.46MB/s] 


Extracting ./datasets\cifar-10-python.tar.gz to ./datasets


In [50]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truch')

In [51]:
class ConNeuNet(nn.Module):
    def __init__(self):
        super(ConNeuNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.dense1 = nn.Linear(16*5*5, 150)
        self.dense2 = nn.Linear(150, 100)
        self.dense3 = nn.Linear(100, 50)
        self.dense4 = nn.Linear(50, 10)

    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(-1, 16*5*5)
        out = F.relu(self.dense1(out))
        out = F.relu(self.dense2(out))
        out = F.relu(self.dense3(out))
        out = self.dense4(out)
        return out
        


In [91]:
model = ConNeuNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [95]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for index, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (index + 1) % 10 == 0:
            print(f'epoch: {epoch + 1}, step = {index+1}, loss = {loss}')
            

epoch: 1, step = 10, loss = 1.8276256322860718
epoch: 1, step = 20, loss = 2.220271587371826
epoch: 1, step = 30, loss = 1.9852821826934814
epoch: 1, step = 40, loss = 1.3126046657562256
epoch: 1, step = 50, loss = 2.1200358867645264
epoch: 1, step = 60, loss = 1.8238911628723145
epoch: 1, step = 70, loss = 1.3446953296661377
epoch: 1, step = 80, loss = 1.7248256206512451
epoch: 1, step = 90, loss = 2.046236991882324
epoch: 1, step = 100, loss = 2.102754592895508
epoch: 1, step = 110, loss = 1.9302780628204346
epoch: 1, step = 120, loss = 1.823427677154541
epoch: 1, step = 130, loss = 1.3998123407363892
epoch: 1, step = 140, loss = 2.0967490673065186
epoch: 1, step = 150, loss = 1.8228923082351685
epoch: 1, step = 160, loss = 1.5538749694824219
epoch: 1, step = 170, loss = 1.6224029064178467
epoch: 1, step = 180, loss = 1.5306265354156494
epoch: 1, step = 190, loss = 2.2759480476379395
epoch: 1, step = 200, loss = 2.0539345741271973
epoch: 1, step = 210, loss = 1.45480215549469
epoch: 

In [96]:
correct = 0
total = 0
for index, (images, labels) in enumerate(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    predictions = model(images)
    _, predictions = torch.max(predictions, dim=1)
    print(predictions, labels)
    total += len(predictions)
    correct += torch.tensor(predictions == labels).sum().item()
print(f'accuracy : {correct / total * 100}')

tensor([6, 0, 0, 0]) tensor([3, 8, 8, 0])
tensor([4, 5, 5, 6]) tensor([6, 6, 1, 6])
tensor([4, 0, 0, 9]) tensor([3, 1, 0, 9])
tensor([4, 7, 9, 6]) tensor([5, 7, 9, 8])
tensor([5, 5, 0, 4]) tensor([5, 7, 8, 6])
tensor([9, 0, 0, 9]) tensor([7, 0, 4, 9])
tensor([4, 4, 6, 4]) tensor([5, 2, 4, 0])
tensor([9, 4, 4, 4]) tensor([9, 6, 6, 5])
tensor([4, 4, 0, 5]) tensor([4, 5, 9, 2])
tensor([4, 9, 9, 5]) tensor([4, 1, 9, 5])
tensor([0, 4, 5, 6]) tensor([4, 6, 5, 6])
tensor([0, 9, 6, 9]) tensor([0, 9, 3, 9])
tensor([4, 4, 0, 2]) tensor([7, 6, 9, 8])
tensor([6, 4, 0, 2]) tensor([0, 3, 8, 8])
tensor([5, 5, 2, 4]) tensor([7, 7, 4, 6])
tensor([4, 4, 5, 9]) tensor([7, 3, 6, 3])
tensor([4, 6, 0, 0]) tensor([6, 2, 1, 2])
tensor([4, 7, 4, 4]) tensor([3, 7, 2, 6])
tensor([8, 0, 9, 4]) tensor([8, 8, 0, 2])
tensor([8, 2, 5, 0]) tensor([9, 3, 3, 8])
tensor([0, 9, 6, 4]) tensor([8, 1, 1, 7])
tensor([4, 5, 5, 0]) tensor([2, 5, 2, 7])
tensor([5, 0, 8, 4]) tensor([8, 9, 0, 3])
tensor([8, 9, 4, 4]) tensor([8, 6,

C:\Users\Phoenix\AppData\Local\Temp\ipykernel_1084\3664048620.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  correct += torch.tensor(predictions == labels).sum().item()


tensor([2, 9, 9, 2]) tensor([5, 9, 8, 0])
tensor([4, 0, 0, 4]) tensor([7, 9, 8, 2])
tensor([4, 4, 9, 4]) tensor([7, 6, 9, 4])
tensor([5, 9, 4, 4]) tensor([3, 9, 6, 4])
tensor([4, 6, 4, 9]) tensor([7, 6, 5, 1])
tensor([5, 0, 8, 0]) tensor([5, 8, 8, 0])
tensor([4, 9, 0, 9]) tensor([4, 0, 5, 5])
tensor([9, 6, 8, 8]) tensor([1, 1, 8, 9])
tensor([2, 9, 5, 0]) tensor([0, 3, 1, 9])
tensor([4, 8, 4, 4]) tensor([2, 2, 5, 3])
tensor([5, 9, 4, 0]) tensor([9, 9, 4, 0])
tensor([6, 8, 0, 9]) tensor([3, 0, 0, 9])
tensor([6, 0, 6, 6]) tensor([8, 1, 5, 7])
tensor([9, 0, 0, 4]) tensor([0, 8, 2, 4])
tensor([4, 0, 4, 4]) tensor([7, 0, 2, 3])
tensor([4, 5, 0, 9]) tensor([6, 3, 8, 5])
tensor([9, 4, 4, 2]) tensor([0, 3, 4, 3])
tensor([0, 4, 6, 0]) tensor([9, 0, 6, 1])
tensor([0, 9, 0, 0]) tensor([0, 9, 1, 0])
tensor([7, 8, 8, 4]) tensor([7, 9, 1, 2])
tensor([4, 0, 5, 4]) tensor([6, 9, 3, 4])
tensor([6, 0, 0, 4]) tensor([6, 0, 0, 6])
tensor([4, 6, 5, 9]) tensor([6, 6, 3, 2])
tensor([4, 8, 9, 4]) tensor([6, 1,